In [1]:
bins = 64
bin_list = list(range(bins))
bin_list = [str(bin).zfill(len(str(64))) for bin in bin_list]
bin_list

['00',
 '01',
 '02',
 '03',
 '04',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28',
 '29',
 '30',
 '31',
 '32',
 '33',
 '34',
 '35',
 '36',
 '37',
 '38',
 '39',
 '40',
 '41',
 '42',
 '43',
 '44',
 '45',
 '46',
 '47',
 '48',
 '49',
 '50',
 '51',
 '52',
 '53',
 '54',
 '55',
 '56',
 '57',
 '58',
 '59',
 '60',
 '61',
 '62',
 '63']

In [2]:
bin_id = "01"

bin_id.count("0")

1

## Gather run times for a single repetition

In [1]:
import os
import pandas as pd

In [48]:
# gather benchmarks for steps that are shared between all error rates
def gather_benchmarks(tool, rep, prefixes):
    dir_path = "../100kb/benchmarks/rep" + str(rep) + "/" + tool
    total_runtimes = []
    prefix_tuple = tuple(prefixes)
    with os.scandir(dir_path) as it:
        for entry in it:
            if (entry.name.endswith(prefix_tuple)) and entry.is_file():
                #print(entry.name)
                benchmark = pd.read_csv(entry.path, sep="\t")
                rep_runtime = benchmark["s"].iloc[0]
                continue
    it.close()

    return rep_runtime

# steps that are repeated for each error rate
def gather_er_benchmarks(tool, rep, prefixes):
    dir_path = "../100kb/benchmarks/rep" + str(rep) + "/" + tool
    total_runtimes = []
    prefix_tuple = tuple(prefixes)
    # summation of run-times from a single repetition
    for er in error_rates:
        rep_er_runtimes = []
        with os.scandir(dir_path) as it:
            for entry in it:
                if (entry.name.endswith(prefix_tuple) or entry.name.endswith("e" + str(er) + ".txt")) and entry.is_file():
                    #print(entry.name)
                    benchmark = pd.read_csv(entry.path, sep="\t")
                    rep_er_runtimes.append(benchmark["s"].iloc[0])
        it.close()
        print(rep_er_runtimes)
        total_runtimes.append(sum(rep_er_runtimes))
    return total_runtimes

In [45]:
rep = 0
n = 2
error_rates = [0, 0.025, 0.05, 0.075, 0.1]

outfile = "../100kb/table1.tsv"

## Average run times over all repetitions

In [46]:
valik_split_tuple = ("valik", ["split_ref.txt"], "valik-split")
valik_build_tuple = ("valik", ["build.txt"], "valik-build")

shared_tuple_list = [valik_split_tuple, valik_build_tuple]

valik_search_tuple = ("valik", [], "valik-search")
distributed_stellar_tuple = ("dream_stellar", ["files.txt"], "distributed-stellar")
stellar_tuple = ("stellar", "stellar")

with_errors_tuple_list = [valik_search_tuple, distributed_stellar_tuple, stellar_tuple]

In [47]:
data = {'error_rate' : error_rates}
df = pd.DataFrame(data)

for method in shared_tuple_list:
    for rep in list(range(n)):
        runtime = gather_benchmarks(method[0], rep, method[1])
        df["rep" + str(rep)] = runtime
    
    col = df.loc[: , "rep0":"rep" + str(n - 1)]
    df[method[2]] = col.mean(axis=1).round(2)
    df = df[df.columns.drop(list(df.filter(regex='rep')))]

for method in with_errors_tuple_list:
    for rep in list(range(n)):
        runtimes = gather_er_benchmarks(method[0], rep, method[1])
        df["rep" + str(rep)] = runtimes
    
    col = df.loc[: , "rep0":"rep" + str(n - 1)]
    df[method[2]] = col.mean(axis=1).round(2)
    df = df[df.columns.drop(list(df.filter(regex='rep')))]

    
df["DREAM-Stellar"] = df["valik-split"] + df["valik-build"] + df["valik-search"] + df["distributed-stellar"]
df["DREAM-Stellar"] = df["DREAM-Stellar"].round(2)

df

valik-split

valik-build

valik-search

[0.4358]
[0.4323]
[0.4419]
[0.5246]
[5.8274]
[0.4344]
[0.4338]
[0.4424]
[0.5203]
[5.7909]
distributed-stellar

[0.1682, 0.1507, 0.1529, 0.1611, 0.1675, 0.1699, 0.1646, 0.1555, 0.1756, 0.1618, 0.1703, 0.1624, 0.162, 0.1675, 0.1083, 0.1565, 0.1553, 0.1821, 0.1666, 0.1777, 0.1692, 0.17, 0.1645, 0.1779, 0.1657, 0.1712, 0.1558, 0.1736, 0.1735, 0.1696, 0.1795, 0.1697, 0.1584, 0.1628, 0.1705, 0.1711, 0.1617, 0.1614, 0.1685, 0.1672, 0.1775, 0.1694, 0.1757, 0.1879, 0.8068, 0.1765, 0.1706, 0.1823, 0.1689, 0.1641, 0.1657, 0.1454, 0.1728, 0.147, 0.1731, 0.1871, 0.1577, 0.1612, 0.1649, 0.1615, 0.1649, 0.1626, 0.1026, 0.4361, 0.1682, 0.1681]
[0.1696, 0.1378, 0.1507, 0.1725, 0.1604, 0.1696, 0.179, 0.1598, 0.1711, 0.1635, 0.1606, 0.1519, 0.1537, 0.1674, 0.1718, 0.1713, 0.185, 0.1702, 0.1665, 0.0104, 0.1621, 0.1633, 0.1622, 0.1616, 0.1561, 0.1772, 0.1619, 0.188, 0.1678, 0.1562, 0.1578, 0.1797, 0.1881, 0.1668, 0.1677, 0.1642, 0.1467, 0.1762, 0.17, 0.1758, 0.1648, 

,error_rate,valik-split,valik-build,valik-search,distributed-stellar,stellar,DREAM-Stellar
0,0.000,0.11,0.19,0.44,11.94,0.26,12.68
1,0.025,0.11,0.19,0.43,11.80,0.20,12.53
2,0.050,0.11,0.19,0.44,12.04,0.24,12.78
3,0.075,0.11,0.19,0.52,11.70,0.31,12.52
4,0.100,0.11,0.19,5.81,13.87,0.33,19.98
